<a href="https://colab.research.google.com/github/ThiagoMueller/csci2470labs/blob/main/2470_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSCI 1470 Lab 01:** Introduction to Machine Learning
---
***important***:
- Before starting the lab please copy this notebook into your own google drive by clicking on "File" and "Save a copy in drive"
- If you want to work locally, make sure that you also download lab1utils.py in the same folder as this notebook
---

In this lab, we will be introducing the basic idea behind machine learning. You will also get to build two machine learning models by yourself. In doing so, you will get familiar with the process of training an ML model from the gradients of its loss function.

The goal of this all this lab is to catch students up to where everybody is on a level playing field. **we hope you find this lab useful! :-)**

**Make sure to get all questions checked off by your TA to get credit for this lab!**

**HINT:**
- Search "Check-" in browser search (cmd+f mac, ctrl+f windows)
- You can also open the "table of contents" from the side menu on the left side of your screen

In [10]:
import os
import sys

isColab = "google.colab" in sys.modules
# this also works:
# isColab = "COLAB_GPU" in os.environ

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

    # TODO: replace ? with the path to your copy of the lab
    colab_path = ("/content/drive"
        + "?")
    sys.path.append(colab_path)

In [13]:
import numpy as np
# import plotly.graph_objects as go
# from sklearn.linear_model import LinearRegression
# import itertools

# make sure that lab1utils.py is either in the same folder as this notebook
# or that its path is included in the system path variable.
# lab1utils.py is a bunch of visualization stuff that you don't need to know right now
import lab1utils

## Conceptual Background

### The Motivation for Machine Learning

In many previous courses, you have had to implement various algorithms to find "solutions". For example:
- You'd like to take a user's math equation and spit out a calculator's solution.
- You'd like to take a graph specification and output the shortest path from one location to another.
- You want a user to run your program and get the text "Hello World!" in the console.

There are many clever algorithms out there that are able to do a lot of things, both very specific and very general. However, there are some problems that may be a bit too hard to code up a good solution for. For example, **how do you identify whether an image is a cat or a dog?** Humans can definitely do it, since you've seen many examples of animals and probably have an internal understanding of what they look like. But how would a computer, which merely has access to a large space of pixels, do it?

It turns out that **machine learning** is a great way of developing such algorithms, and can be applied when we have examples to train off of. The basic paradigm shift can be described as follows:

 - **Normal Computation:** Inputs and Function -> Outputs
 - **Machine Learning:**
   - **Supervised:** Inputs and Outputs -> Function that approximately maps the inputs to the outputs
   - **Unsupervised:** Inputs -> Associations and patterns of the input data

This course will really mostly focus on supervised machine learning formulations in the earlier parts, so we're just going to introduce this for now. In supervised machine learning, we have a lot of data with various properties. Some of the properties are easy to get, while others are not so simple. The idea is that we want to predict some of the harder properties in the data.

### A Simple Problem Formulation

Let's imagine that you have this magical function that can tell if the animal in the image is a cat or a dog $f: \mathbf{X} \mapsto Y$:

\begin{align*}
\mathbf{x} &\in  \{ 0, 1, \cdots, 255 \}^{H \times W \times 3}
    & \text{ (pixel values of an image)}\\
y &\in \{\text{cat}, \text{dog}\}
    & \text{ (true label of an image)} \\
f(\mathbf{x}) &= y
    & \text{ (true function from image to label)}\\
\end{align*}

**Supervised Learning**
- **Goal:** Discover the function $y = f(\mathbf{x})$ so you can use it.
- **Problem:** $f$ usually cannot be found analytically or is too complicated.
- **Solution:** Find an **approximation** $\hat{y} = h(\mathbf{x})$ that is similar to $y$.

- **Limitations:**
    - The data is often-times generated from the real world, so we have limited observations.
        - **Solution**: Train $h$ on a subset of known data, and then see how it performs on held-out a testing subset of data.
    - We might have no idea of the underlying data relationships.
        - **Solution:** We make an assumption on kind of function $h$ could be and test out our hypothesis by fitting and evaluating it.
- **Examples:** linear regression, quadratic discriminant analysis, k-nearest neighbors, support vector machine, random forest, etc

**Unsupervised Learning**
+ The goal is to find the associations and patterns in a set of input variables $\{\mathbf{x}_{1}, \mathbf{x}_{2}, \cdots, \mathbf{x}_{n} \}$.
+ There is no target variable $y$; instead, we have target function classes, such as a clustering function.
+ **Examples:** principal component analysis, k-means clustering, etc

In this lab, we will begin with the supervised learning algorithms, which is a bit easier to understand conceptually.

### Loss Functional

**How do we actually find the prediction function $h: X \to Y$ in supervised learning?**
 + We find the **target function** that minimizes the **loss functional**
 + The **loss functional** $L: H \to \mathbb{R}$ measures the distance between the model $h$ and the ground truth $f$.
    + $H$ is the space of all candidate functions.
        + In theory, these can be anything. However, in practice people make assumptions about what these functions can be and bind them to forms that are tuneable by some parameters $\theta$. As a very simple example, $H$ could be defined by the function $h_\theta(x) = x\theta$ and reduce their search space to a selection of $\theta$.
    + The target function is the member $h$ that minimizes the distance $\mathrm{argmin}_{h \in H}{L(h)}$, and is the closest approximation to the true $f$.
        + If we reduce our search space to $h_\theta$, this becomes $\mathrm{argmin}_{\theta}L(h_\theta)$.
 + The goal is to make $\hat{y} = h_{\pmb{\theta}}(\mathbb{x})$ as close to $y = f(\mathbf{x})$ by tweaking the parameters $\pmb{\theta} = \{\theta_1, \theta_2, \cdots, \theta_n\}$.

Here is overall process:
1. Construct a model that incorporates the prediction function $h_{\pmb{\theta}}$.
2. Keep tweaking the parameters $\pmb{\theta} = \{\theta_1, \theta_2, \cdots, \theta_n\}$.
3. Find the set of parameters that minimizes the loss functional $L(h_{\pmb{\theta}})$.
4. Evaluate the performance of the model with a new set of data that the model has not seen yet.

**However, you don't really have a method of evaluating the loss functional in the exact analytic form**.
 + You only have a finite number of the input-target variable pairs $\{(\mathbf{x}_{1}, y_{1}), (\mathbf{x}_{2}, y_{2}), \cdots, (\mathbf{x}_{n}, y_{n}) \}$
 + Thus, you calculate the **empirical loss function** $\mathbb{E}[L(\theta, \mathbf{x}, y)]$ instead.
 + Remember that the values of $\mathbf{X}$ and $y$ are fixed, because they are members of the training dataset $\mathbb{X}_{train}, \mathbb{Y}_{train}$
 + The empirical loss function is only a function of the parameters $\theta$.
 + **Then the problem is how to find the optimal set of parameters $\theta$ that minimizes the empirical loss function**.

Here are some examples of empirical loss functions:
 - In a regression problem
   - The empirical loss function is often the mean squared error $L(h) = \mathbb{E}[(h(X) - Y)^2]$.
   - The target function is the regression function $r(\mathbf{x}) = \mathbb{E}[Y | \mathbf{X} = \mathbf{x}]$.
 - In a classification problem:
   - The empirical loss function is often the misclassification probability $L(h) = \mathbb{P}[h(\mathbf{X}) \neq Y]$.
   - The target function is $G(\mathbf{x}) = \mathrm{argmax}_{c} \mathbb{P}[Y = c | \mathbf{X} = \mathbf{x}]$.


We may also choose to incorporate a validation dataset. The formal role distribution is:
- Use $\mathbb{X}_{train}, \mathbb{Y}_{train}$ to directly inform $\theta$.
- Use $\mathbb{X}_{valid}, \mathbb{Y}_{valid}$ to suggest generalizability (and indirectly inform architecture of $h$).
- Use $\mathbb{X}_{test}, \mathbb{Y}_{test}$ to suggest generalizability (and verify that validation dataset didn't screw up model selection).

### Optimization and Gradient Descent

If the loss function $L(\theta, \mathbb{X}_{train}, \mathbb{Y}_{train})$ is simle enough, then it is easy to minimize it. We simply find all the points where the partial derivative $\frac{\partial L}{\partial \theta} = 0$ and take the smallest one as the global minimum.

- **Problem:** In many cases and certainly in deep learning, the loss function is too complicated and it is practically impossible to solve $\frac{\partial L}{\partial \theta} = 0$.
- **Solution:** If we can evaluate $\frac{\partial L}{\partial \theta}$, the derivative of loss with respect to the tuneable parameters $\theta$, we can train our model to approach a minimum.

**What do we do when the loss function is too complicated?**
- We use the technique called the **gradient descent**.
- We begin from a random set of parameters $\theta_{\text{initial}}$
- We repeat updating the parameters with the **gradient of the loss function** $\theta_{\text{new}} = \theta_{\text{old}} - \eta \frac{\partial L}{\partial \theta}$.
- Here $\eta$ is a carefully-chosen number called the **learning rate**.
  
This method is what we use to train more complicated machine learning models such as the deep learning models.

We will revisit gradient descent with actual examples later in this Notebook, but for now, here is the takeaway.
> The gradient of the loss as a function of input, output, and prediction!

Here is how gradient descent works.
+ If the gradient is **positive** (${L}$ and $\theta$ are **positively correlated** at current configuration), ***decreasing $\theta$ decreases ${L}$***.
+ If the gradient is **negative** (${L}$ and $\theta$ are **negatively correlated** at current configuration), ***increasing $\theta$ decreases ${L}$***.

So... **why not just shift $\theta$ by the negative gradient?** When the parameter setting has a lot of impact on the loss w/ current param configurations, its gradient will have a large magnitude and so it will be shifted significantly and vice versa. This seems like a good plan (and is the simplest version of gradient descent).

### Supervised Learning Examples

<details><summary><b>Click here for more examples of supervised learning problems</b></summary>

- **Predict position of a ball after $t$ seconds.**
  - $h : \mathbb{R} \to \mathbb{R}$. Input might be lower-bounded.
  <details><summary><b>Basic Regression</b></summary> $\mathbb{R}$ is the set of all real numbers, which include positive and negative decimals and irrational numbers (i.e. $\pi, e, \sqrt{2}$). This is a natural regressional (numerical) formulation: The output is a numerical value that represents some continuous distribution. </details>
  - Maybe minimize Euclidean Distance $L(h, x, y) = ||h(x) - y||_2$.

+ **Predict the job type (out of 8 possible jobs) of employees based on the pay, number of years worked, and years in school**
  + $h : \mathbb{R}^3 \to [0,1]^8$
  <details><summary><b>One-Hot Encoding Representation</b></summary> A set raised to a power $n$ is set of $n$-element combinations of the set. For example, $\{0,1\}^2 = \{(0,0), (0,1), (1,0), (1,1)\}$ where each pair of numbers is technically a tuple. The problem with 8 arbitrary job types is that there is not a natural progressive relationship among them. Let's say that for a particular example, your model predicts job 3 (mechanic), and the right answer is job 7 (programmer). Does increasing your prediction to job 3.5 or job 4 (a secretary) make your prediction any better? No. However, let's say that you're now associating each class as a probability and are predicting $[0, 0, 0.9, 0, 0, 0, 0.1, 0]$. We can then try to transition this into a correct prediction pretty easily without any cross-category conflicts. </details>
  + $h$ probably contains softmax to conform to discrete probability distribution.
  + Consider using $L(h, x, y) = \text{CE}(h(x), y)$.

- **Predict whether an image contains a cat or a dog.**
  - $h : \{0,\cdots,255\}^{H \times W \times 3} \to [0, 1]$
  <details> <summary><b>BIG Input Space</b></summary> Note how the image space is very big. For a 100x100 image with 3 channels (and 255 intensity options per channel), that's ~30 million possible inputs. </details>
  <!-- - $h : \{0,\cdots,255\}^{H \times W \times 3} \to \sigma(\mathbb{R}^2) = [0, 1]$ for softmax function $\sigma$ -->

<!-- - Colorize a greyscale photo based on a reference pic.  -->
  <!-- - $h : \{0,\cdots,255\}^{H \times W \times (3+1)} \to \{0,\cdots,255\}^{H \times W \times 3}$ -->

+ **Pick a move for a robot that can make one of 10 motions**.
  + $h : \text{State Abstraction Space} \to [0,1]^{10}$
  <details> <summary><b>What's The Input Space?</b></summary> Some problems make it really hard to define an input space. What level of abstraction would you even use? For chess, do you want to use a grid representation? Maybe a graph representation? How about the image representation when viewed from a particular camera? </details>
<!-- - Generate the next word in a sentence after $n$ words.  -->
  <!-- - $h : D^n \to D$ for a dictionary $D$ with a lot of english words.  -->
</details>

### Further Notes on Supervised ML

<details> <summary> <b> Click here if you need more nerdy details </b> </summary>

- In the above discussions, $x$ is an element of the set $\mathbb{X}$ which is a set of realizations of the distribution $X$ (and the same goes for $y$ etc.). However, we will begin referring to $\mathbb{X}$ and $\mathbb{Y}$ as vectors when it becomes useful (this is how they are stored in memory anyways).

- $h(x) = \hat{y}$ is a common and convenient notation. These symbols can be called the 'prediction' or the 'hypothesis'.

- $\mathbb{Y}$ can be referred to as the "ground truth"... even though it could have plenty of elements that are misrepresentative or low-quality...

- $\mathbb{X, Y} \subset D(\mathbb{X}), D(\mathbb{Y})$ : This is trivially true, but it's worth mentioning. The observations $\mathbb{X}$ and $\mathbb{Y}$ are a subset of all possible inputs/outputs that span over some mathematical domain.

- $\mathbb{X, Y} \sim X, Y$ : $X$ and $Y$ come from of some probability distributions $X,Y$.

- $X$ and $Y$ are interdependent : $X$ and $Y$ depend on each other such that $Y$ can be predicted by $X$ with some amount of confidence.

- The true function $f: X \to Y$ such that $f(\mathbb{X}) = \mathbb{Y}$ is an assumption that we can make from the interdependence between $X$ and $Y$. In reality, whenever you are making an observation, you are drawing samples from the joint distribution $X \times Y$.

</details>

---

## Regression Problem: Associating Inputs and Outputs

Enough of theory. Let's actually do something!

For this exercise, we will be modeling a basic **regression task**; that is, the task of finding an interdependance between two sets of values:
- **Independent Variables**: The *inputs* that are pulled from some space.
- **Dependent Variables**: The *outputs* that depend on the inputs.

**For this task, we will do the following:**
- Create a toy dataset with known distribution to train our models on.
- Formulate the mean function as a 'model' and get a sense for the lingo.
- Make a basic ***linear regression*** model to show the basics.
- Extend the model to reason with ***generalized regression*** problems.

### Toy Dataset

Every machine learning project starts with collecting data, and that's also where we will begin. Real data relationships are extremely complex and usually not known, but in general associated variables have two components:
- **Dependence** $\mathbb{E}[Y | X = \mathbf{x}]$: the components of $Y$ that is explained by $X$.
- **Noise** $\xi = p - \mathbb{E}[Y | X = \mathbf{x}]$: The components of $Y$ that is **not** explained by $X$.

We will make some synthetic data, making sure to maintain both dependance and noise by generating from random distributions.

- Let $x_1, x_2$ be generated from a multi-normal distribution with non-diagonal covariance matrix.
    - Simply put, this makes $x_1$ and $x_2$ somewhat interdependent; a diagonal cov matrix would make them independent of each other.
- Let $y$ be generated from a normal distribution with mean $\mu_y$ being a complex function of $x_1$ and $x_2$ and standard deviation $\sigma_y \neq 0$.
    - This means that $\mathbb{E}[Y | X = \mathbf{x}]$ is a complex function which gets obscured by random noise quantity $\xi \sigma_y$.

\begin{align*}
(x_1, x_2) & \sim \mathrm{MultiNormal}(
\mu = [1, 2], \,
\mathrm{cov} =
\begin{bmatrix}
6 & 3 \\
3 & 6
\end{bmatrix}
) \\
\\
y & \sim \mathrm{Normal}(\mu = \mu_y,\, \sigma = \sigma_y) \\
& \mu_y = 0.8x_1 + 0.2x_2 - 0.05x_{1}^2 - 0.04x_{1}x_{2} + 20 \\
& \sigma_y = 1 \\
\end{align*}

#### [Check-off #1] Generate Data
- Read the NumPy API guide on the [multivariate normal generator](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.multivariate_normal.html) and complete the `generate_X` function below.
- Read the NumPy API guide on the [normal generator](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.normal.html) and complete the `generate_y` function below.

In [14]:
## DO NOT CHANGE ANY OF THE DEFAULT VALUES
def generate_X(n_obs, seed=42):
    """Draws random samples from the parameterized 2D multi-normal distribution"""
    mean = (1, 2)
    cov = ((6, 3), (3, 6))

    rng = np.random.default_rng(seed = seed)
    X = rng.multivariate_normal(mean, cov, size=n_obs)              ## TODO
    return X

def generate_y(X, seed=42):
    """Draws random samples from the parameterized normal distribution"""
    weights = (20.0, 0.8, 0.2, -0.05, -0.04, 0.0)
    stdev = 1.0

    b0, b1, b2, b11, b12, b22 = weights
    x1, x2 = X[:, 0], X[:, 1]
    mean = b0 + b1*x1 + b2*x2 + b11*(x1**2) + b12*(x1*x2)

    rng = np.random.default_rng(seed = seed)
    y = rng.normal(loc=mean, scale=stdev, size=X.shape[0]).reshape(-1, 1)         ## TODO
    return y

X = generate_X(300)
print(f"shape of inputs: {X.shape}")
print(f"first five inputs: \n{X[:5]}\n")

y_true = generate_y(X)
print(f"shape of outputs: {y_true.shape}")
print(f"first five outputs: \n{y_true[:5]}")

shape of inputs: (300, 2)
first five inputs: 
[[ 1.62731266  0.07988226]
 [-1.7438992   1.56000442]
 [ 6.73360831  4.54393296]
 [ 1.11612605  1.34149306]
 [ 2.08040181  0.99087946]]

shape of outputs: (300, 1)
first five outputs: 
[[21.4849366 ]
 [17.83365782]
 [23.5551678 ]
 [21.97958628]
 [19.61260147]]


The result should look like this.
```
shape of inputs: (300, 2)
first five inputs:
[[ 1.62731266  0.07988226]
 [-1.7438992   1.56000442]
 [ 6.73360831  4.54393296]
 [ 1.11612605  1.34149306]
 [ 2.08040181  0.99087946]]

shape of outputs: (300, 1)
first five outputs:
[[21.4849366 ]
 [17.83365782]
 [23.5551678 ]
 [21.97958628]
 [19.61260147]]
```

We can visualize the data in the 3D scatter plot here.

In [15]:
fig_data = lab1utils.visualize_data(X, y_true)
fig_data.show()

**Please feel free to drag the 3D plot with your mouse pointer and inspect the data from various angles**.

### Mean Predictor Model

So, now that we have our data all gathered up, let's make the simplest model possible.

In this case, we already know the true distribution $X \times Y$, so we don't even need machine learning in the first place. However, we'll pretend that we have no idea how this data was generated because – in many real cases – **we don't know the underlying distribution of our data.** All we have is some number of realizations $\mathbb{X}$ and $\mathbb{Y}$ pulled from the distribution.

The best that we can do in real cases is to make the assumption that there is some interdependence between $\mathbb{X}$ and $\mathbb{Y}$, and try to find the relation $y = f(x_1 , x_2)$ by using machine learning. Hopefully, if we do it right, our prediction function $h(x_1 , x_2)$ will have some predictive power that generalizes from our limited observable set to the set of unobserved points that will be encountered later.

As a baby step, let's first make the mean predictor model, which will always return the same number no matter the values of the independent variables $\mathbf{x}$, and that number is going to be the mean of all target variable $y$ in the training dataset. Just look at the code cell below and see how ridiculously simple the model is.

In [16]:
class MeanPredictor:
    def __init__(self):
        self.mean = 0

    def __str__(self):
        return f"mean predictor model, mean = {self.mean}"

    def __call__(self, X):
        y_pred = self.mean * np.ones_like(X[:, [0]])
        return y_pred

    def fit(self, X, y_true):
        self.mean = np.mean(y_true)

The mean of the target variable $y$ in the training data is about 20.578 and that's what this model always returns.

In [17]:
mp_model = MeanPredictor()
mp_model.fit(X, y_true)
X_checkoff = np.array([[1, 1], [2, 3], [1.5, 2]])
print(f"mp_model(X_checkoff) = \n{mp_model(X_checkoff)}")

mp_model(X_checkoff) = 
[[20.63104932]
 [20.63104932]
 [20.63104932]]


This model is your "Hello, World" to machine learning. Just like "Hello, World", the goal is not to understand the model itself, but to understand everything else around it. In this case, that "everything else" is going to be the interface to interact with the model.

Here, the `ModelWrapper` is a wrapper around a model to make interacting with the model easier, and by easier we mean that you can interact with the model with fewer lines of code.

In [19]:
class ModelWrapper:

    _loss_dict = {
        "MAE" : lambda y_true, y_pred: np.mean(np.abs(y_true - y_pred)),        # mean abosolute error
        "MSE" : lambda y_true, y_pred: np.mean((y_true - y_pred)**2),           # mean squared error
        "RMSE": lambda y_true, y_pred: np.sqrt(np.mean((y_true - y_pred)**2)),  # root mean squared error
    }

    def __init__(self, model):
        self.model = model

    def __str__(self):
        return f"wrapper around the following model:\n{self.model}"

    def _get_loss_fn(self, loss_key): ## <-- NOT IMPORTANT!!
        '''If loss_key is a function, return it.
            Otherwise, retrieve the loss function by keyword (i.e. 'MAE', 'MSE')
        '''
        if isinstance(loss_key, str):   return self._loss_dict[loss_key]
        elif callable(loss_key):        return loss_key
        else: raise ValueError('Loss function/key must be specified')

    def loss(self, X, y_true, loss="MSE"):
        y_pred = self.model(X)
        loss_fn = self._get_loss_fn(loss)
        return loss_fn(y_true, y_pred)

    def train(self, X, y_true, **kargs):
        train_return = self.model.fit(X, y_true, **kargs)
        return train_return

    def predict(self, x):
        return self.model(x)

The benefit of using a wrapper is that you can use the model with fewer lines of code, and the interface that the wrapper provides will always be the same no matter how complicated your model is. This benefit might not be obvious to you with a simple mean-predicting model like this one, but it will be handy if you use a more complicated model.

In [20]:
# new instance, not yet trained
mp_model2 = MeanPredictor()
mp_model_wrapper = ModelWrapper(mp_model2)
print(f"Prediction before training = \n{mp_model_wrapper.predict(X_checkoff)}\n")

# loss before training
print(f"Loss before training = {mp_model_wrapper.loss(X, y_true)}\n")

# train
mp_model_wrapper.train(X, y_true)
print(f"Model is trained\n")

# Prediction after training
print(f"Prediction after training = \n{mp_model_wrapper.predict(X_checkoff)}")

# loss after training
print(f"Loss after training = {mp_model_wrapper.loss(X, y_true)}\n")

Prediction before training = 
[[0.]
 [0.]
 [0.]]

Loss before training = 429.6543796913059

Model is trained

Prediction after training = 
[[20.63104932]
 [20.63104932]
 [20.63104932]]
Loss after training = 4.014183656015185



You can see that the loss is significantly smaller after the training. Now let's visualize our model's prediction.

In [21]:
fig_mean = lab1utils.visualize_data(X, y_true)
go_mean = lab1utils.get_mean_go(mp_model2.mean)
fig_mean.add_trace(go_mean)
fig_mean.show()

Now you must feel like you can do better than this, and you definitely can! Just not with this architecture.

This mean predictor model has only one parameter $m$, and by setting it equal to the mean of all $y$ values in the training set, you have already minimized the mean squared loss.

The true optimal function is $f(x) = \mathbb{E}[Y | X = \mathbf{x}] = \mu_y(x)$. Recall that this is a very complex function, so... how can we go about fitting a more complex function – maybe one that assumes that $y$ is a function of $x$ – to this kind of data?

<!-- If you want a better performance from your model, you have to change its very architecture, because you cannot make any further improvements by changing its parameter.  -->

<!-- \begin{align*}
\mathrm{MSE}(m) &=
\frac{1}{n}\sum_{i=1}^{n}{(y_{\text{true}}^{(i)} - y_{\text{pred}}^{(i)})^2} \\
&=
\frac{1}{n}\sum_{i=1}^{n}{(y_{\text{true}}^{(i)} - m)^2} \\
\end{align*} -->

## Intro to Linear Models

In our mean model:
- There was only one parameter, which was extremely easy to set.
- Any particular configuration of it will always output just a single value regardless of the input.

We will need to go to more powerful models with more parameters if we want to actually learn more powerful correlations, and this is where the more standard regression models come in.

A standard ***linear regression*** model makes the assumption that some linear scaling of the inputs contributes directly to the output. In other words, for some vector of weights $\theta$:
> $\mathbb{E}[Y | X = \mathbf{x}] \text{ is apporimately } \hat y = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots$


Let's use the exact same toy dataset again. The regression function would be the following.

> $r(\mathbf{x}) = \mathbb{E}[Y | X = \mathbf{x}] =  0.8x_1 + 0.2x_2 - 0.05x_{1}^2 - 0.04x_{1}x_{2} + 20$

You may notice that the standard linear regression model still isn't powerful enough to represent the true distribution's regression. If you fit a linear regression model on this data, you'll actually get something like this:

In [22]:
# fitting linear regression model using the provided lab1utils library
coef_optimized, bias_optimized = lab1utils.linreg_answer_key(X, y_true)
go_answer_key = lab1utils.get_linreg_go(coef_optimized, bias_optimized)

# visualization
fig_linreg = lab1utils.visualize_data(X, y_true)
fig_linreg.add_trace(go_answer_key)
fig_linreg.show()

Obviously, the linear regression model does not fully capture the non-linearity in the data, but it is doing a pretty good job in the more densely populated areas, and it is so much better than the mean predictor model. Now, let's build a linear regression model by ourselves.

### Linear Regression Model Architecture

Now let's make our own implementation of the linear regression algorithm. As you have probably learned before, linear regression assumes that the observed target variables is a random draw from the normal distribution, of which the mean is given by an affine function of the independent variables and the standard deviation is an unknown parameter.

\begin{align*}
\mu &= \beta_{0} + \beta_{1} x_{1} + \beta_{1} x_{2} \\
y &\sim \mathrm{Normal}(\mu, \sigma)
\end{align*}

The goal of linear regression is to find the best set of coefficients ($\beta_1$ and $\beta_2$) and bias ($\beta_0$), and there are several different ways to do it, in both the frequentist and the Bayesian approaches. Here, we will do it with gradient descent, which is quite an unusual way method to fit a linear regression model, but it would be a good preparation for the deep learning models later in the course. Right now, don't worry about the methods `get_gradients`, `gradient_descent` and `optimize`, and just implement the `__call__` method.

#### [Check-off #2] Implement the `__call__` method

In [25]:
class LinearRegression_v1:
    def __init__(self, n_vars, seed = None):
        self.coef = np.zeros(n_vars).reshape(-1, 1) # Initialize a np array of shape (n_var, 1)
        self.bias = np.float64(0.0)
        # Did you know that np.float64 has many more convenient methods
        # that vanila Python float does not have?

        if seed: # equivalent to "if seed is not None", but shorter and faster
            rng = np.random.default_rng(seed = seed)
            self.coef = rng.normal(self.coef, scale = 1.0)
            self.bias = np.float64(rng.normal(self.bias, scale = 1.0))

    def __str__(self):
        coef_string = np.array2string(self.coef,
                                      precision = 6,
                                      prefix = " - coefficients = ")
        return (f"linear regression model\n" +
                f" - coefficients = {coef_string}\n" +
                f" - bias = {self.bias:.6f}")

    def __call__(self, X):
        """
        TODO: Implement the call function.
        - use np.matmul() to directly implement the matrix formulation in the LaTeX cells below
        - you can alternatively use np.dot() or np.sum(), but it is not really a direct
          implementation of the matrix formulation below and we will secretly judge you.
        - you are not allowed to use any explicit loop statements like a for-loop or a while-loop.
        """
        y_pred = np.matmul(X, self.coef) + self.bias                                     ## TODO
        return y_pred

    def get_gradients(self, X, y_true):
        """you will implement it later"""
        pass

    def gradient_descent(self, X, y_true):
        """you will implement it later"""
        pass

    def fit(self, X, y_true):
        """it will be given to you later"""
        pass

Since we are implementing in Python, we will want to take advantage of matrix operations to speed up computation by vectorizing our solution. Libraries like numpy will create pre-compiled utilities that perform matrix operations in a lower-level language than Python, which allows extremely fast iteration and good parallelization.

To help us facilitate this, the following operation scheme will be used.

<!-- In our model, we would like to be able to make predictions in a batch by using matrix operations.
By "batching", we mean that:
- **We do not** want to calculate the prediction on the sets of input variables one input-pair at a time, i.e. computing $\hat{y}^{(1)} = \beta_{0} + \beta_{1} x_{1}^{(1)} + \beta_{2} x_{2}^{(1)}$, then $\hat{y}^{(2)}$, and so on.
- **Rather, we would like to calculate the predictions $\hat{y}^{(1)}, \hat{y}^{(2)}, \cdots, \hat{y}^{(n)}$ all at once** from all observations of input variables $\left\{ (x_{1}^{(1)}, x_{2}^{(1)}),\, (x_{1}^{(2)}, x_{2}^{(2)}),\, \cdots \, (x_{1}^{(n)}, x_{2}^{(n)}) \right\}$ together.

The way that we will batch the data is through the matrix formulation, for which different people use slightly different versions, but the version that we are going to use is the following.  -->

\begin{align*}
\textrm{coefficients}: \pmb{\beta} = \begin{bmatrix}
\beta_1 \\
\beta_2
\end{bmatrix} \quad
\textrm{bias}: \beta_0 \quad
\textrm{inputs}&: \mathbf{X} = \begin{bmatrix}
x_{1}^{(1)} & x_{2}^{(1)} \\
x_{1}^{(2)} & x_{2}^{(2)} \\
\vdots & \vdots \\
x_{1}^{(n)} & x_{2}^{(n)}
\end{bmatrix}
\end{align*}

\begin{align*}
\textrm{predictions}: \mathbf{Y_\text{pred}} &= \mathbf{X} \pmb{\beta} + \beta_{0} \mathbf{1}
\ \ =\ \ \begin{bmatrix}
x_{1}^{(0)} & x_{2}^{(0)} \\
x_{1}^{(1)} & x_{2}^{(1)} \\
\vdots & \vdots  \\
x_{1}^{(n)} & x_{2}^{(n)}
\end{bmatrix}
\begin{bmatrix}
\beta_{1} \\
\beta_{2} \\
\end{bmatrix} +
\begin{bmatrix}
\beta_{0} \\
\beta_{0} \\
\vdots \\
\beta_{0}
\end{bmatrix}
\ \ =\ \ \begin{bmatrix}
\beta_{0} + \beta_{1} x_{1}^{(0)} + \beta_{2} x_{2}^{(0)} \\
\beta_{0} + \beta_{1} x_{1}^{(1)} + \beta_{2} x_{2}^{(1)} \\
\vdots \\
\beta_{0} + \beta_{1} x_{1}^{(n)} + \beta_{2} x_{2}^{(n)}
\end{bmatrix}
\end{align*}

Run the following cell after you have implemented the `__call__` method.

In [26]:
LRv1_model_core = LinearRegression_v1(2, seed = 42)
LRv1_model_wrapper = ModelWrapper(LRv1_model_core)
print(LRv1_model_wrapper)

X_checkoff = np.array([[1, 1], [2, 3], [1.5, 2]])
y_checkoff = LRv1_model_wrapper.predict(X_checkoff)
print(f"\n", f"outputs_checkoff = \n{y_checkoff}")

wrapper around the following model:
linear regression model
 - coefficients = [[ 0.304717]
                   [-1.039984]]
 - bias = 0.750451

 outputs_checkoff = 
[[ 0.01518417]
 [-1.76006696]
 [-0.8724414 ]]


The results from the previous cell should look like this:
```
wrapper around the following model:
linear regression model
 - coefficients = [[ 0.304717]
                   [-1.039984]]
 - bias = 0.750451

outputs_checkoff =
[[ 0.01518417]
 [-1.76006696]
 [-0.8724414 ]]

```

Now, the model has not been trained yet. However, before we go out training the model, we need a loss function.

### Linear Regression Loss Function

In the case of linear regression, the default choice of loss function is the mean squared error. In our case, it can be expressed as the following.

\begin{align*}
L(h) &= \mathbb{E}[(y - h(\mathbf{x}))^2] \\
&= \frac{1}{n}\sum_{i=1}^{n}{\big(y^{(i)} - (\beta_{0} + \beta_{1} x_{1}^{(i)} + \beta_{2} x_{2}^{(i)})\big)^2}
\end{align*}

The frequentist approach to finding the set of paremeters ($\beta_0$, $\beta_1$, and $\beta_2$) is to find the minimum point of the empirical loss function, where the gradient will be the zero vector.

\begin{align*}
\nabla{L(h)} &=
(\frac{\partial L(h)}{\partial \beta_0}, \,
\frac{\partial L(h)}{\partial \beta_1}, \,
\frac{\partial L(h)}{\partial \beta_2}) \\
&= (0,\, 0,\, 0)
\end{align*}

In this lab, we will find the minimum point **without setting the partial derivaties to zero and solving them directly**. Instead, we will use an iterative method to gradually get closer to the minimum, which is called the **gradient descent** method.  

Here is a visualization of the loss function for you. We couldn't really visualize all four dimensions of $(L,\, \beta_{0},\, \beta_{1},\, \beta_{2})$. By the limit of our biology and our flat computer screens, we can only see in 3D. So, we decided to visualize the 3D intersection of the 4D hyperparabola where $\beta_{0}$ is set to be its optimal value (which we computed for you, so don't worry about it).

In [30]:
b1_mesh, b2_mesh, mse_mesh = lab1utils.get_loss_mesh(X, y_true, bias_optimized)
fig_loss = lab1utils.visualize_loss_function(b1_mesh, b2_mesh, mse_mesh)
fig_loss.show()

### Optimizing via Gradient Descent

The supervised machine learning objective we've been discussing involves minimizing the loss evaluation for the data we are training on. We haven't coded up a routine for that yet though. In our case, the prediction function is $h(\mathbf{x}) = \beta_{0} + \beta_{1} x_{1} + \beta_{1} x_{2}$, but we have not yet found the set of parameters $(\beta_{0},\, \beta_{1},\, \beta_{2})$ that minimizes the loss function, and that's where gradient descent chimes in.

Obviously, we want to shift our weights towards a good configuration. One pretty instinctive approach is to shift them in a way that makes the loss decrease. This is the idea behind gradient descent. The idea here is that we can change the weights in such a way that we incrementally minimize the loss evaluation.

- **Gradient:** Generalization of slope into higher dimensions. It's the rate of change when going in a speciifc direction.
- **Descent:** To go down. In this case, this is referring to the gradient of the loss function.

How can we do that? **With some calculus**!

1. We begin by randomly initializing the parameters.

$$\beta_{j} = \beta_{j, \text{initial}}, \quad j \in \{0, 1, 2\}$$

2. We update the parameters with the gradients of the loss function.

$$\beta_{j, \text{new}} = \beta_{j, \text{old}} - \eta \frac{\partial L}{\partial \beta_{j}}, \quad j \in \{0, 1, 2\}$$

3. Repeat updating the parameters until the loss value converges to its minimum.

Here, $\eta$ is called the **learning rate**, which is a carefully chosen number to ensure that the paramters will converge to the minimum point.

In our case, the gradient for the bias is the following.

\begin{align*}
\frac{\partial L(h)}{\partial \beta_{0}} &=
\frac{\partial}{\partial \beta_{0}} \mathbb{E}[(y - h(\mathbf{x}))^2] \\
&= \frac{\partial}{\partial \beta_{0}} \left(
\frac{1}{n}\sum_{i=1}^{n}{\big(y^{(i)} - \beta_{0} - \beta_{1} x_{1}^{(i)} - \beta_{2} x_{2}^{(i)}\big)^2}
\right) \\
&= - \frac{2}{n} \sum_{i=1}^{n} \big( y^{(i)}  - {\hat{y}}^{(i)} \big)
\end{align*}

In our case, the gradients for the coefficients are the following.

\begin{align*}
\frac{\partial L(h)}{\partial \beta_{1}}  
&= \frac{\partial}{\partial \beta_{1}} \left(
\frac{1}{n}\sum_{i=1}^{n}{\big(y^{(i)} - \beta_{0} - \beta_{1} x_{1}^{(i)} - \beta_{2} x_{2}^{(i)}\big)^2}
\right) \\
&= - \frac{2}{n} \sum_{i=1}^{n} \big( y^{(i)}  - {\hat{y}}^{(i)} \big) x_{1}^{(i)}
\\
\frac{\partial L(h)}{\partial \beta_{2}}  
&= - \frac{2}{n} \sum_{i=1}^{n} \big( y^{(i)}  - {\hat{y}}^{(i)} \big) x_{2}^{(i)}
\end{align*}

#### [Check-off #3] Implement the gradient descent

Implement the `get_gradients` method and the `gradient_descent` below. You can take a hint by looking at the `fit` method.

In [28]:
class LinearRegression_v2(LinearRegression_v1):
    # def __init__(self, n_vars, seed = None): ## Already Implemented in v1
    #     self.coef
    #     self.bias
    # def __str__(self):                       ## Already Implemented in v1
    # def __call__(self, X):                   ## Already Implemented in v1

    def get_gradients(self, X, y_true):
        """
        - returns the gradients for the coefficients and the bias
        - do not use a for-loop or a while-loop
        - use only vectorized operations with NumPy
          - use np.mean() and be careful with the axis of vectorization
        - grad_coef should be a NumPy array of shape (n_vars, 1),
          which is the same shape as self.coef
        - grad_bias should be a np.float64 variable
        """
        differences = y_true - self(X)

        ## TODO
        grad_coef = -2 * np.mean(differences * X, axis=0).reshape(-1, 1)
        grad_bias = -2 * np.mean(differences)
        return grad_coef, grad_bias

    def gradient_descent(self, grad_coef, grad_bias, learning_rate):
        """
        - does not return anything.
        - just updates self.coef and self.bias
        """
        ## TODO
        self.coef -= learning_rate * grad_coef
        self.bias -= learning_rate * grad_bias

    def fit(self, X, y_true,
                 epochs = 40,
                 learning_rate = 0.05,
                 print_frequency = 10):

        ## Keeping track of the trajectories
        coef_trajectory = [self.coef.copy().reshape(-1,1)]
        bias_trajectory = [self.bias.copy()]
        loss_trajectory = [np.mean((y_true - self(X))**2)]

        for current_epoch in range(1, 1+epochs):
            ## Main bulk
            grad_coef, grad_bias = self.get_gradients(X, y_true)
            self.gradient_descent(grad_coef, grad_bias, learning_rate)

            ## Keeping track of the trajectories
            coef_trajectory += [self.coef.copy().reshape(-1,1)]
            bias_trajectory += [self.bias.copy()]
            loss_trajectory += [np.mean((y_true - self(X))**2)]

            ## Printing out the progress
            if current_epoch%print_frequency == 0:
                print(f"epoch = {current_epoch}, loss = {loss_trajectory[-1]:.6f}")

        trajectories = (
            np.stack(coef_trajectory),  ## Convert list of many np.arrays into
            np.stack(bias_trajectory),  ##   a single large np.array
            np.array(loss_trajectory)   ## Convert list into np.array
        )

        return trajectories

Run the cell below after implmenting the gradient descent.

In [29]:
LRv2_model_core = LinearRegression_v2(2, seed = 42)
LRv2_model_wrapper = ModelWrapper(LRv2_model_core)
print(LRv2_model_wrapper)

gradients_initial = LRv2_model_wrapper.model.get_gradients(X, y_true)

print("")
print(f"gradients on coefficients: \n{gradients_initial[0]}")
print(f"gradient on bias: {gradients_initial[1]}")

wrapper around the following model:
linear regression model
 - coefficients = [[ 0.304717]
                   [-1.039984]]
 - bias = 0.750451

gradients on coefficients: 
[[ -57.7751128]
 [-100.8285632]]
gradient on bias: -43.28252455275603


If everything is correct, the result should look like this.

```
wrapper around the following model:
linear regression model
 - coefficients = [[ 0.304717]
                   [-1.039984]]
 - bias = 0.750451

gradients on coefficients:
[[ -57.7751128]
 [-100.8285632]]
gradient on bias: -43.28252455275603
```

#### [Check-off #4] Size of learning rate

Repeat runing the next cell multiple times with different values of `learning_rate` to visualize its effect. Again, we cannot really visualize 4D, so we decided to visualize the 3D intersection of the 4D hyperparabola where $\beta_{0}$ is already optimal. Then, discuss the following cases with the TA.

1. What happens when the learning rate is too small, like `learning_rate = 0.001`?
2. What happens when the learning rate is too big, like `learning_rate = 0.072`?
3. How should we find the perfect learning rate?

When the learning rate is too small, the model updates its parameters very slowly. This means it will take a very long time to converge to the minimum loss.

If the learning rate is too large, instead of converging, the loss starts to increase with each update, and the model never finds the optimal set of parameters.

A common approach is to use a range of learning rates and observe the model's performance.

In [41]:
# Let's re-initialize the model every time you run the cell again
LRv2_model_core = LinearRegression_v2(2, seed = 42)
LRv2_model_wrapper = ModelWrapper(LRv2_model_core)

# for the purpose of visualization
# beta_0 is set, so only beta_1 and beta_2 are changing
LRv2_model_wrapper.model.bias = bias_optimized

# Training the model
coef_trajectory, bias_trajectory, loss_trajectory = \
    LRv2_model_wrapper.train(X, y_true,
                             epochs = 40,
                             learning_rate = 0.05, ## try different values for the learning rate
                             print_frequency = 10)

## Visualizing the model
b1_mesh, b2_mesh, mse_mesh = lab1utils.get_loss_mesh(X, y_true, bias_optimized)
fig_gradients = lab1utils.visualize_loss_function(b1_mesh, b2_mesh, mse_mesh)
go_gradients, go_gradients_2D = lab1utils.get_gradients_go(coef_trajectory, loss_trajectory,
                                                           offset=0.5)
fig_gradients.add_trace(go_gradients)
fig_gradients.add_trace(go_gradients_2D)
fig_gradients.show()

epoch = 10, loss = 1.264749
epoch = 20, loss = 1.254556
epoch = 30, loss = 1.251233
epoch = 40, loss = 1.250103


### Visualize the Training Process

Finally, you can see how the model gets updated at every epoch.

In [42]:
# re-initialize the model
LRv2_model_core = LinearRegression_v2(2, seed = 42)
LRv2_model_wrapper = ModelWrapper(LRv2_model_core)
print(LRv2_model_wrapper)

# train the model again
coef_trajectory, bias_trajectory, loss_trajectory = \
    LRv2_model_wrapper.train(X, y_true,
                             epochs = 40,
                             learning_rate = 0.05, ## try different values for the learning rate
                             print_frequency = 10)

# visualize
fig_process = lab1utils.visualize_trajectory(X, y_true, coef_trajectory, bias_trajectory)
fig_process.show()

wrapper around the following model:
linear regression model
 - coefficients = [[ 0.304717]
                   [-1.039984]]
 - bias = 0.750451
epoch = 10, loss = 59.488765
epoch = 20, loss = 21.059766
epoch = 30, loss = 7.990925
epoch = 40, loss = 3.543614


## Next Steps

Some of you have probably noticed that **we have not divided up our data in this lab**. In many machine learning applications, you evaluate your model with the set of data that has not been used in the training process. So you would typically divide your data into the training set and the testing set, and you would use the training set to optimize the model parameters and the testing set to evaluate the model's performance. If your model has some hyperparameters, then you would divide up the data into the training set, validation set, and the testing set, and you would use the validation set to fit the hyperparameters.

We didn't divide up the dataset in this lab, **because we wanted this lab to be a baby step for you**. However, starting from the first homework assignment, you will have separate training set and testing set.

----

# **2470 Section**: Implementing Logistic Regression From Scratch

**This section is optional for 1470 students.**

We have explored linear regression to make predictions on a continuous dependent variable. Now, we will look at a classification problem where our dependent variable is discrete or categorical. We will make predictions with the logistic regression model.

##Classification: Linear Regression vs Logistic Regression

Linear regressions makes predictions on a continuous range. For example, if we
want to predict sunny or rainy weather, to use the linear regression model, we would have to convert this categories into numerical identifiers (1 for rain, 0 for sun). Could you identify a problem with using linear regression in this case? How could we interpret a predicted weather outcome of 0.7? How about 1.5?

For binary classification, we need an ML model that gives us a bound outcome between 0 and 1 that we can interpret as the probablility of a target event. This is not possible by fitting a linear function to the data.

##Sigmoid function

Logistic regression is centered around the logistic function that describes
the probability of event $x$ based on the trainable parameters $\beta_{0}$ and $\beta_{1}$.

$$p\left(x\right) = \frac{1}{1 + e^{-\left(\beta_{0} + \beta_{1}x\right)}}$$

## Binary Cross-Entropy Loss

Going back to the weather example (1 for rain, 0 for sun), we want from our model:

  * $p(x)$ to be close to 1 as possible when it rains. Ideally:
  $$\Pi_{y_{i} = 1} p\left(x_{i}\right) = 1$$
  * $1 - p(x)$ to be close to 1 as possible when it is sunny.
  $$\Pi_{y_{i} = 0} \left(1 - p\left(x_{i}\right)\right) = 1$$

Then, we want to maximize:

$$\Pi_{y_{i} = 1} \space p\left(x_{i}\right)  \times  \Pi_{y_{i} = 0} \left(1 - p\left(x_{i}\right)\right)$$

This is called **Maximum Likelihood Estimation**, and it is the method used to calculate the parameters for a logistic regression function. Note that it is very similar to log-loss or cross-entropy loss. In that case, we include a negative sign in front of the expression because we want to **minimize loss** (as opposed to maximize likelihood of the prediction based on the data).

We rewrite the maximum likelihood expression to arrive to the following formula:

$$\Pi_{s} \space p\left(x_{i}\right) ^{y_{i}} \times \left(1 - p\left(x_{i}\right)\right)^{1-y_{i}} = $$


$$\sum_{i=1}^{n} y_{i}\log\left(p\left(x_{i}\right)\right) + (1-y_{i})\log(1-p(x_{i}))$$

The binary cross-entropy loss, then, seeks to maximize the likelihood of correct prediction given the data, by minimizing for all datapoints:

$$-\left[y_{i}\log\left(p\left(x_{i}\right)\right) + (1-y_{i})\log(1-p(x_{i}))\right]$$

## Implementing Logistic Regression

In this lab, you will be implementing a logistic regression model to be trained on a breast cancer dataset provided by Sklearn. ([Sklearn]((https://https://scikit-learn.org/stable/getting_started.html/)) is a python library popular for machine learning and data analysis. It provides many basic machine learning models, sample dataset like MNIST, and other helper functions such as train-test splitor and model evaluators). You will use the binary cross-entropy as the loss function and train the model using gradient descent.

One difference here is we want challenge you to derive the gradient formula yourself. Consult the *Optimizing via Gradient Descent* section of the lab for how to derive the gradients using calculus. We will ask you to show you work and explain the process during check-off as well, so **be prepared**.

In [43]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [50]:
class Logistic_Regression():
    def __init__(self, n_vars, seed = 42):
        self.weights = np.zeros(n_vars).reshape(-1, 1)
        self.bias = np.float64(0.0)

        if seed:
            rng = np.random.default_rng(seed = seed)
            self.weights = rng.normal(self.weights, scale = 1.0)
            self.bias = np.float64(rng.normal(self.bias, scale = 1.0))

    def __call__(self, X):
        ## TODO: Implement the call function
        logit = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(logit)


        return y_pred # the probability of y being 1

    def sigmoid(self, logit):
        ## TODO: Implement sigmoid function
        logit = np.clip(logit, -500, 500)
        sigmoid = 1 / (1 + np.exp(-logit))

        return sigmoid

    def loss(self, y_true, y_pred):
        # TODO: Implement binary cross-entropy loss function.
        # If you encounter "log on zero" error message, consider adding a constant epsilon inside the log functions
        epsilon = 1e-9
        loss = -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))

        return loss

    def accuracy(self, y_true, y_pred):
      return np.sum(y_pred == y_true)/len(y_pred)

    def get_gradients(self, X, y_true):
        """
        Returns the gradients for the weights and the bias.
        - do not use a for-loop or a while-loop
        - use only vectorized operations with NumPy
          - use np.mean() and be careful with the axis of vectorization
        - grad_weights should be a NumPy array of shape (n_vars, 1),
          which is the same shape as self.weights
        - grad_bias should be a np.float64 variable
        """
        # TODO: Derive the loss function with respect to parameter B_{1}
        # to obtain the formula for the gradient of the weight.
        # Then, return the mean of all gradients.

        # Similarly, derive the loss function with respect to parameter B_{0}
        # to obtain the formula for the gradient of the bias.
        # Then, return the mean of all gradients.

        # TODO:

        y_pred = self.__call__(X)
        error = y_pred - y_true.reshape(-1, 1)
        grad_weights = np.mean(error * X, axis=0).reshape(-1, 1)
        grad_bias = np.mean(error)
        return grad_weights, grad_bias


    def gradient_descent(self, grad_weights, grad_bias, learning_rate):
        """
        Update the trainable parameters with the learning rate and the gradients.
        """
        # TODO:
        self.weights -= learning_rate * grad_weights
        self.bias -= learning_rate * grad_bias

    def fit(self, X, y_true,
                 epochs = 2000,
                 learning_rate = 0.02,
                 print_frequency = 100):
        print("Start training of logistic regression model.")

        for current_epoch in range(1, 1+epochs):
            ## Main bulk
            grad_coef, grad_bias = self.get_gradients(X, y_true)
            self.gradient_descent(grad_coef, grad_bias, learning_rate)

            if current_epoch%print_frequency == 0:
                print(f"""epoch = {current_epoch}, loss = {self.loss(y_true,
                          self.__call__(X))}, accuracy = {self.accuracy(
                              y_true, self.__call__(X))}""")
        print("End training of logistic regression model.")

In [51]:
# Load and extract data.
dataset = datasets.load_breast_cancer()
X = dataset.data
y = dataset.target
# Divide data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

dataset_size = X.shape[0]
feature_size = X.shape[1]

print(f"number of data samples: {dataset_size}")
print(f"number of features: {feature_size}")

number of data samples: 569
number of features: 30


In [52]:
LogR_model = Logistic_Regression(feature_size)
LogR_model.fit(X_train, y_train)
print(f'Testing Accuracy Score: {accuracy_score(y_test, LogR_model(X_test).astype(int))}')

Start training of logistic regression model.
epoch = 100, loss = 7.971001987313595, accuracy = 215.53387533875338
epoch = 200, loss = 9.319161069521636, accuracy = 160.53116531165313
epoch = 300, loss = 9.238157391816268, accuracy = 162.44986449864498
epoch = 400, loss = 9.267999862082402, accuracy = 161.81029810298102
epoch = 500, loss = 9.19374970177384, accuracy = 165.64769647696477
epoch = 600, loss = 9.381507304999998, accuracy = 157.97289972899728
epoch = 700, loss = 9.319161069521634, accuracy = 160.53116531165313
epoch = 800, loss = 9.2878085327261, accuracy = 161.81029810298102
epoch = 900, loss = 9.254163726178795, accuracy = 161.81029810298102
epoch = 1000, loss = 9.318155580523008, accuracy = 159.89159891598916
epoch = 1100, loss = 9.296264148718832, accuracy = 159.89159891598916
epoch = 1200, loss = 9.617374565383061, accuracy = 146.46070460704607
epoch = 1300, loss = 9.687363619730892, accuracy = 143.90243902439025
epoch = 1400, loss = 9.735005067805897, accuracy = 142.62

#### [2470 (1470-optional) Check-off] You model should reach at least 80% accuracy

## Acknowledgements & Sources

This lab is originally written by HTA Vadim Kudlay with neural network models as the examples of machine learning models for the Spring 2022 Semester. Then, it was rewritten to use the linear regression model by TA Yeunun Choo and editted by HTA Vadim Kudlay for the Fall 2022 Semester. The lab was updated by
TA Tanadol Lamlertprasertkul and TA Shirley Loayza Sanchez for the Fall 2022 semester.

For more, please consult the textbook [The Elements of Statistical Learning](https://hastie.su.domains/ElemStatLearn/).